## Lets Create a Price Adjuster

So, we have an idea, to simplify the work required to find nearby rentals that are
similar to the new rentals that I am adding and attempt to create a price calculator
that will provide me with the best expected price from those results.   

With that in mind lets start the process.  

### Setting up

So the first thing that we are going to do is setup our imports, so that we have all the tools
we are going to work with loaded (don't need to load later)

In [ ]:
import pandas as pd
import numpy as np

from pprint import pprint

pd.options.display.max_rows = 4000
pd.options.mode.chained_assignment = None

### First Analyze the Data

So to start lets look over the data that we have received, first we will
need to load the data so that we can manipulate it later. 

In [ ]:
data = pd.read_csv('../data/listing.csv')

At this point lets do a quick check of the data for row count and even first row to sample the data.  

In [ ]:
print('Row Count: {}'.format(len(data)))
print('-'*20)
print(data.iloc[0])

So the column that we are trying to determine for future entries is the price column.  Knowing that
we need to determine which other columns we can use to create features.  

Since all modeling requires that the data be represented as a numeric value, and since we are limited
on time, we are going to focus only on columns that already have a defined numeric representation.  

So we are going to ignore columns that are:  

* Text
 * URL
 * Short/Long Descriptions
* Date
* Lat/Long

That leaves use with a number of columns still, but for our first attempt lets just use a few columns:  

* bathrooms
* bedrooms
* beds
* accommodates
* region_id
* price

In [ ]:
columns = ['bathrooms', 'bedrooms', 'beds', 'accommodates', 'region_id', 'price']
data.dropna(axis=0, how='any', subset=columns, inplace=True)

subset = data[columns]
pprint(subset.dtypes)

So looking over the columns, it seems that the price column is the only one that is not already
in a numeric type, so lets start by looking at the price value in the first 10 rows to determine
what the data looks like.  

In [ ]:
print(subset['price'][:5])

Ok, so it looks like it is formatted with a `$` in front, so lets convert the column value and
translate it to a numeric type.  

In [ ]:
data['price'] = data['price'].str.replace('$', '')
data['price'] = data['price'].str.replace(',', '')
data['price'] = data['price'].astype(float)

subset = data[columns]
print(subset['price'][:5])
print(subset.dtypes)

---

### Modeling

Alright so at this point we have some data to work with and have it formatted in a fashion that we
can start creating some models.  

For our first model we are going to go with a simple regression model using **K-Nearest Neighbors**

This algorithm simply takes the supplied feature vector and finds the _k_ closest examples to the
selected vector (nearest neighbors).  In the case of regression it will take the values from each
of the _k_ neighbors and calculate the average.  

For our setup we are going to use the Euclidean Distance to determine how closely related
two vectors are.   

#### Euclidean Formula

The formula is defined below.  

p - current vector
q - comparing vector

\begin{equation*}
distance = \sqrt{(p_1-q_1)^2 + (p_2-q_2)^2 + \cdots + (p_n-q_n)^2}
\end{equation*}

or shorter form

\begin{equation*}
distance = \sqrt{\sum_{i=0}^{n}(p_i-q_i)^2}
\end{equation*}

So now lets go through an example of how we can use this to calculate the distance.  

In this example we are going to use only two of the features (bathrooms, bedrooms) and compare
the distance for the first two rows.  

In [ ]:
features = ['bathrooms', 'bedrooms']
p = subset[features].iloc[0]
q = subset[features].iloc[1]
print('p')
print('-'*20)
print(p)
print('q')
print('-'*20)
print(q)

In [ ]:
distance = np.sqrt(np.sum(np.square(p-q)))
print(distance)

So lets check our math to make sure this is correct.  

\begin{equation*}
2.0 = \sqrt{(1-1)^2 + (1-3)^2}
\end{equation*}

Looks good to me, this means that the distance between the first two entries is 2.0.  

Lets go ahead and calculate the distance between our selected entry and the rest of the dataset.  

In [ ]:
p = subset.iloc[0]
subset = subset[1:]
subset['distance'] = subset[features].apply(lambda q: np.sqrt(np.sum(np.square(p[features]-q))), axis=1)
print(subset['distance'].value_counts().head(5))
print(subset.iloc[-1])

So it looks like we actually have 1264 neighbors... that are the closest (as close as possible)
and if we wanted to select _k_ of them, we would need to make sure that we are `unbiased` in our
selection, so lets sort the distance values (and randomize them when they are close).  

After doing that we can select the top _k_ rows (in this case lets go with the 5 closest neighbors). 

In [ ]:
random_order = np.random.permutation(len(subset))
subset = subset.loc[random_order]
#subset.sort_values(['distance'], inplace=True)
neighbors = subset[:5]
print(subset.iloc[-1])

Now that we have our neighbors we can compare the mean price of the neighbors
and use that to determine how close we are to the actual price of our selected
entry.  


In [ ]:
mean_price = neighbors['price'].mean()
p_price = p['price']
print(f'estimated price: {mean_price}')
print(f'actual price: {p_price}')

So we have an estimated price and an actual price that we generated using the 5 nearest neighbors.  
Now we need to take these values and calculate the error that we introduced.  

This can be done by using an algorithm named the _Mean Absolute Error_ or (MAE).  The MAE algorithm
is defined below.  

\begin{equation*}
error = \dfrac{|g_0 - a_0| + |g_1 - a_1| + \cdots + |g_n - a_n|}{n}
\end{equation*}

or the simplier form

\begin{equation*}
error = \dfrac{\sum_{i=0}^{n}{|(g_i - a_i)|}}{n}
\end{equation*}

In our case n = 1 so we can simplify it to

\begin{equation*}
error = |m - p|
\end{equation*}


In [ ]:
error = np.abs(mean_price - p_price)
print(error)

Now there is one thing that we will want to do when calculating an error, we actually want values
that are farther from the true value to be penalized more.  This can be accomplished using the
MSE or _Mean Squared Error_.  

\begin{equation*}
error = \dfrac{\sum_{i=0}^{n}{(g_i - a_i)^2}}{n}
\end{equation*}

Of course this means that the value losses its representation and we can no longer consider it to 
represent a dollar, although that can be fixed just by taking the sqrt of the MSE. 

\begin{equation*}
error = \sqrt{\dfrac{\sum_{i=0}^{n}{(g_i - a_i)^2}}{n}}
\end{equation*}


---

#### Sklearn implementation

At this point we have gone over our own version of the *K Nearest Neighbor* using the
Euclidean distance algorithm and determined the error.  These algorithms have actually
all been created before (and optimized).  So instead of re-inventing the wheel, lets 
see how we can quickly create a model using the sklearn framework.  

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

subset = data[columns]
p = subset.iloc[0]
subset = subset[1:]

model = KNeighborsRegressor(algorithm='brute', n_neighbors=5)
# fit our model (calculate the distances), specify the value to use for regression
model.fit(subset[features], subset['price'])
# test our model out 
#  Note: we are reshaping the input so that as per sklearn's requirement for single samples
predictions = model.predict(p[features].values.reshape(1, -1))
error = mean_squared_error([p['price']], predictions)
print(error)

Now this shows the MSE for a single sample, the top one.  What we really want to do is to 
take a subset of our data and split it into two groups, training data and test data.  

The rule of thumb for seperating this data is to use ~70% as the training data, and the
other ~20% as the testing data.  

So here lets create our train and testing data. 

In [ ]:
from sklearn.utils import shuffle
from math import floor

subset = data[columns]
subset = shuffle(subset)

data_size = len(subset)
split_index = floor(data_size*.7)
training_set = data[:split_index]
test_set = data[split_index:]

Now lets re-run our above code using the training_set and test_set.  

In [ ]:
model = KNeighborsRegressor(algorithm='brute', n_neighbors=5)
# fit our model (calculate the distances), specify the value to use for regression
model.fit(training_set[features], training_set['price'])
# test our model out 
#  Note: we are reshaping the input so that as per sklearn's requirement for single samples
predictions = model.predict(test_set[features])
error = mean_squared_error(test_set['price'], predictions)
print(error)

So we have our error on our testing and training set, lets create a function that we can use
to quickly run this. 

In [ ]:
def test_model(features, k):
    model = KNeighborsRegressor(algorithm='brute', n_neighbors=k)
    model.fit(training_set[features], training_set['price'])
    predictions = model.predict(test_set[features])
    error = mean_squared_error(test_set['price'], predictions)
    return error

At this point we have a way to adjust our features, as well as our single hyper parameter.  Lets
try it out by using the same features that we have been using (bedrooms, bathrooms) but by adjusting
the _k_ value (from 1 to 10).   

In [ ]:
error_values = []
k_range = list(range(1, 11))
for k in k_range:
    error_values.append(test_model(features, k))
pprint(error_values)

While this might be a nice list... it isn't the best for us to quickly see how well our model is working
with the different hyper parameter value, so lets try to visual the results.  

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(k_range, error_values)
plt.show()

Alright, so right now it looks like the data is still getting better when we get to the point of
using `k=10`.  

Lets try it with more k values, from 1 to 40 this time.  

In [ ]:
error_values = []
k_range = list(range(1, 41))
for k in k_range:
    error_values.append(test_model(features, k))

plt.scatter(k_range, error_values)
plt.show()

Interesting, so from what is shown above, it seems that a k value of 28 seems to be our best bet.  

Lets go ahead and create another function, one that does a k value search but allows you to use different
features and just returns the best k value and error it sees.  

In [ ]:
def k_search(features, max_k=40):
    min_result = None
    for k in range(1, max_k+1):
        error = test_model(features, k)
        if min_result is None or error < min_result[1]:
            min_result = k, error, np.sqrt(error)
    return min_result

Now lets run some experiments with a few different feature sets.  

* `['bathrooms', 'bedrooms']`
* `['accomodates']`
* `['accomodates', 'region_id']`
* `['bedrooms', 'beds', 'region_id']`
* `['accomodates', 'beds']`
* `['accomodates', 'bathrooms', 'bedrooms']`
* `['accomodates', 'beds', 'bathrooms', 'bedrooms']`

In [ ]:
feature_sets = [
    ['bathrooms', 'bedrooms'],
    ['accommodates'],
    ['accommodates', 'region_id'],
    ['bedrooms', 'beds', 'region_id'],
    ['accommodates', 'beds'],
    ['accommodates', 'bathrooms', 'bedrooms'],
    ['accommodates', 'beds', 'bathrooms', 'bedrooms'],
]

error_values = []
for fs in feature_sets:
    error_values.append(k_search(fs))
    
pprint(error_values)

---

#### Conclusion

So it appears that our best model is one that is built using 'accommodates' as the selected
feature, with a _k_ value of 4.   